**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import pandas as pd
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout

/Users/noemiequere/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e
        
    def get_epsilon(self):
        return self.epsilon

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

At each time step, there are K possible actions (here k=4). The function ```act``` chooses the next action to perform based on either a *explorative* or *exploitative* strategy. In 10% of the cases (random value [0,1)] < epsilon, with epsilon = 0.1), it will choose an *explorative* strategy, by picking a random action instead of the greedy action. In the rest of the cases (probability 1 - ```epsilon```) , it will pick the greedy action, defined by:

\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

```epsilon``` is essential because it prevents the model from overfitting during the training. The fact that epsilon is set at a very low value (0.1) is good so that it allows a strong bias towards exploitation over exploration, favouring choosing the action with the highest q-value over a random action.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #error ==> self.position[-2:, :]
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=11 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```position``` is used to track the position of the rat on the island, that is to say its position on the board, which is defined by an 10x10 grid. In the ```act``` function, the ```position``` is updated depending on the value of x and y, that are changed regarding the action taken by the player. The value in the array corresponding to the position of the rat takes value 1, that is to say if the rat is on the cell of coordinates (3,4), the matrix of dimension (board_size, board_size) will take value 1 at position (3,4). It is then used to update the state. The array has values -1 on the 2-cell edges of the board.

```board``` simply every possible place the area where the rat can be, with coordinates x and y. The cells have different values that corresponds to colors: red is cheese, blue is poison and grey represents an empty cell. Every time the agent moves, the corresponding cell becomes (or stays) grey to indicate whether the rat has eaten the poison, the cheese, or if nothing happened.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        n_action = 4
        action = np.random.randint(0, n_action, size=1)[0]
 
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        ##### FILL IN HERE
        
        #restart a fresh game at each epoch
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        
        while not game_over:
        
            action = agent.act(state)
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random10.mp4'))

Win/lose count 7.5/14.0. Average score (-6.5)
Win/lose count 7.0/14.0. Average score (-6.75)
Win/lose count 7.5/9.0. Average score (-5.0)
Win/lose count 17.5/14.0. Average score (-2.875)
Win/lose count 14.0/19.0. Average score (-3.3)
Win/lose count 11.5/20.0. Average score (-4.166666666666667)
Win/lose count 7.5/20.0. Average score (-5.357142857142857)
Win/lose count 10.5/13.0. Average score (-5.0)
Win/lose count 10.0/10.0. Average score (-4.444444444444445)
Win/lose count 16.5/15.0. Average score (-3.85)
Win/lose count 6.5/7.0. Average score (-3.5454545454545454)
Final score: -3.5454545454545454


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer__


We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

We can write Q recursively to get a Bellman equation.

\begin{equation*}Q^\pi(s,a)=E_{(s', a')\sim p(.|s,a)}[r(s,a) + {\gamma}\sum_{t\leq T}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \> 
\end{equation*}

\begin{equation*}Q^\pi(s,a)=E_{(s', a')\sim p(.|s,a)}[r(s,a) + {\gamma}Q^\pi(s',a')] \> .
\end{equation*}

Where s' and a' are the state and the action in the period following that of s and a.

Given that the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

By plugging this into the Bellman equation, and by exploiting the Markov property:

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

Therefore, taking a mean squared error loss, a plausible objective function is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

We try to choose Q such that it is as close as possible to the optimal strategy.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        self.i = 0

    def remember(self, m):
        
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory[self.i] = m
            self.i += 1
            if self.i >= self.max_memory:
                self.i=0
                
    def random_access(self):
        
        rand_access = np.random.randint(len(self.memory))
        
        return self.memory[rand_access]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        act_pred = self.model.predict(np.reshape(s, (1, 5, 5, self.n_state)))
        
        return np.argmax(act_pred[0])
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
         # Two steps: first memorize the states, second learn from the pool
        

        self.memory.remember([s_, n_s_, a_, r_, game_over_])

        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            #everytime we loop, get a new item from memory
            batch = self.memory.random_access()
            
            #get variables depending on their position in the batch
            state = batch[0]
            next_state = batch[1]
            action = batch[2]
            reward = batch[3]
            game_over = batch[4]
            
            input_states[i] = state # state observed in t
            
            #when game is over target is stored reward
            if game_over_:
                target_q[i, action] = reward 
                
            #otherwise, the reward is changedd
            else:
                ns_reshaped = np.reshape(next_state, (1, 5, 5, self.n_state))
                target_q[i, action] = (reward + self.discount * np.amax(self.model.predict(ns_reshaped)))
    
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        
        return l
         

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)

        max_memory = 500
        batch_size = 50
        grid_size = 10
        
        
        # NN Model
        model = Sequential()
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,2)))
        model.add(Dense(24, input_dim=50, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(4, activation='linear')) #4 is equal to the number of moves
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.5, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/011 | Loss 0.0096 | Win/lose count 5.0/11.0 (-6.0)
Epoch 001/011 | Loss 0.0151 | Win/lose count 4.0/3.0 (1.0)
Epoch 002/011 | Loss 0.0045 | Win/lose count 5.5/6.0 (-0.5)
Epoch 003/011 | Loss 0.0144 | Win/lose count 6.0/4.0 (2.0)
Epoch 004/011 | Loss 0.0071 | Win/lose count 3.5/3.0 (0.5)
Epoch 005/011 | Loss 0.0096 | Win/lose count 6.0/2.0 (4.0)
Epoch 006/011 | Loss 0.0051 | Win/lose count 4.5/2.0 (2.5)
Epoch 007/011 | Loss 0.0115 | Win/lose count 2.5/3.0 (-0.5)
Epoch 008/011 | Loss 0.0062 | Win/lose count 2.5/1.0 (1.5)
Epoch 009/011 | Loss 0.0027 | Win/lose count 3.5/2.0 (1.5)
Epoch 010/011 | Loss 0.0120 | Win/lose count 4.0/7.0 (-3.0)


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        #model.add(MaxPooling2D(pool_size=(1, 1)))
        
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/011 | Loss 0.0013 | Win/lose count 2.5/0 (2.5)
Epoch 001/011 | Loss 0.0111 | Win/lose count 5.5/9.0 (-3.5)
Epoch 002/011 | Loss 0.0018 | Win/lose count 3.0/3.0 (0.0)
Epoch 003/011 | Loss 0.0149 | Win/lose count 5.0/4.0 (1.0)
Epoch 004/011 | Loss 0.0239 | Win/lose count 5.0/5.0 (0.0)
Epoch 005/011 | Loss 0.0159 | Win/lose count 3.5/2.0 (1.5)
Epoch 006/011 | Loss 0.0008 | Win/lose count 4.0/3.0 (1.0)
Epoch 007/011 | Loss 0.0018 | Win/lose count 6.5/5.0 (1.5)
Epoch 008/011 | Loss 0.0018 | Win/lose count 5.0/3.0 (2.0)
Epoch 009/011 | Loss 0.0075 | Win/lose count 5.5/1.0 (4.5)
Epoch 010/011 | Loss 0.0025 | Win/lose count 2.0/0 (2.0)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.0/5.0. Average score (0.0)
Win/lose count 6.5/3.0. Average score (1.75)
Win/lose count 2.0/4.0. Average score (0.5)
Win/lose count 8.0/4.0. Average score (1.375)
Win/lose count 0.5/3.0. Average score (0.6)
Win/lose count 3.5/2.0. Average score (0.75)
Win/lose count 6.0/3.0. Average score (1.0714285714285714)
Win/lose count 2.0/2.0. Average score (0.9375)
Win/lose count 3.5/0. Average score (1.2222222222222223)
Win/lose count 2.5/1.0. Average score (1.25)
Win/lose count 5.5/2.0. Average score (1.4545454545454546)
Final score: 1.4545454545454546
Test of the FC
Win/lose count 5.5/6.0. Average score (-0.5)
Win/lose count 5.5/2.0. Average score (1.5)
Win/lose count 6.0/4.0. Average score (1.6666666666666667)
Win/lose count 5.5/6.0. Average score (1.125)
Win/lose count 8.0/8.0. Average score (0.9)
Win/lose count 4.0/5.0. Average score (0.5833333333333334)
Win/lose count 4.5/3.0. Average score (0.7142857142857143)
Win/lose count 4.5/1.0. Average score (1.0625)

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

The CNN doesn't explore the map very much, and thus is only trained by a few patterns, which impacts negatively its learning. The fully connected (FC) is better in this situation.

When the temperature grows, the score of both models grows as well. The temperature corresponds to the parameter of the binomial distribution which determines the density of poison and cheese on the board. If a poison and a cheese are distributed on the same cell, this cell takes the value 0 (grey). Thus, when the temperature grows, the proportion of cheese and poison items will grow as well, which leads to improved results since a denser board helps the agent train and learn more.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [19]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            
            #decrease the value of epsilon
            agent.set_epsilon(np.float(agent.get_epsilon())*0.995)
            
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True) #state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=10, max_time=500, temperature=0.1)
        
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
        def draw(self,e):
            skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

        def get_frame(self,t):
            b = np.zeros((self.grid_size,self.grid_size,3))+128
            b[self.board>0,0] = 256
            b[self.board < 0, 2] = 256
            b[self.x,self.y,:]=256
            b[-2:,:,:]=0
            b[:,-2:,:]=0
            b[:2,:,:]=0
            b[:,:2,:]=0
        
            b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

            self.to_draw[t,:,:,:]=b
            
#switch to true so that it would work for test
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #error ==> self.position[-2:, :]
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
          
        self.t = self.t + 1
        reward = 0
        
        if train:
            reward = -self.malus_position[self.x, self.y]
            
        self.malus_position[self.x, self.y] = 0.01
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/011 | Loss 0.0106 | Win/lose count 11.5/12.389999999999977 (-0.8899999999999775)
Epoch 001/011 | Loss 0.0072 | Win/lose count 4.98/4.809999999999963 (0.17000000000003723)
Epoch 002/011 | Loss 0.0050 | Win/lose count 6.880000000000002/6.849999999999979 (0.030000000000022453)
Epoch 003/011 | Loss 0.0118 | Win/lose count 2.48/2.8399999999999848 (-0.3599999999999848)
Epoch 004/011 | Loss 0.0031 | Win/lose count 1.47/1.9800000000000015 (-0.5100000000000016)
Epoch 005/011 | Loss 0.0086 | Win/lose count 1.49/1.9600000000000015 (-0.4700000000000015)
Epoch 006/011 | Loss 0.0017 | Win/lose count 4.430000000000001/1.8800000000000014 (2.549999999999999)
Epoch 007/011 | Loss 0.0168 | Win/lose count 3.9400000000000004/4.929999999999961 (-0.9899999999999602)
Epoch 008/011 | Loss 0.0067 | Win/lose count 3.45/1.9400000000000015 (1.5099999999999987)
Epoch 009/011 | Loss 0.0098 | Win/lose count 0.49/2.0000000000000013 (-1.5100000000000013)
Epoch 010/011 | Loss 0.0012 | Win/lose count 4.41000000

In [21]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 0.98/1.9800000000000015. Average score (-1.0000000000000016)
Win/lose count 6.880000000000002/1.8700000000000014. Average score (2.004999999999999)
Win/lose count 6.370000000000002/2.8799999999999817. Average score (2.5000000000000058)
Win/lose count 0.98/1.9900000000000015. Average score (1.622500000000004)
Win/lose count 0.98/4.989999999999961. Average score (0.4960000000000111)
Win/lose count 2.45/1.9600000000000015. Average score (0.495000000000009)
Win/lose count 0.49/2.0000000000000013. Average score (0.20857142857143612)
Win/lose count 2.9400000000000004/2.9499999999999806. Average score (0.18125000000000913)
Win/lose count 1.47/1.9800000000000015. Average score (0.10444444444445236)
Win/lose count 0.98/1.9900000000000015. Average score (-0.006999999999993034)
Win/lose count 2.45/1.9600000000000015. Average score (0.038181818181824416)
Final score: 0.038181818181824416


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***